# Projektarbeit Gruppe 2: Gesundheitsdaten

Sebastian Grab, Josua Klöble, Imanuel Ricker

Konsolidierung aller benötigten Anforderungen innerhalb unseres Use Cases.

https://www.kaggle.com/datasets/sooyoungher/smoking-drinking-dataset

In [ ]:
# Import der benötigten Bibliotheken

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Datentransformation & -exploration

1. Daten laden und verstehen

In [ ]:
# Lokaler Repository Pfad auslesen zum dynamischen Laden unabhängig des Nutzers:
path = str(os.getcwd()).replace("\Projektarbeit", "")

# Lesen der CSV Datei:
dataset = pd.read_csv(path + '\smoking_drinking_dataset_V0.csv')

In [ ]:
# Umbenennen der Spalten:

dataset = dataset.rename(columns={'sex': 'Geschlecht', 'age': 'Alter', 'height': 'Größe', 'weight': 'Gewicht', 'waistline': 'Hüftumfang', 'sight_left': 'Sehkraft_links', 'sight_right': 'Sehkraft_rechts', 'hear_left': 'Hörkraft_links', 'hear_right': 'Hörkraft_rechts', 'SBP': 'Systolischer Blutdruck', 'DBP': 'Diastolischer Blutdruck', 'BLDS': 'Blutleckdetektor', 'tot_chole': 'Totale Cholesterin', 'HDL_chole': 'HDL_Cholesterin', 'LDL_chole': 'LDL_Cholesterin', 'triglyceride': 'Triglycerid', 'hemoglobin': 'Hemoglobin', 'urine_protein': 'Urin_Proteine', 'serum_creatinine': 'Serum_Kreatinin', 'SGOT_AST': 'SGOT_AST', 'SGOT_ALT': 'SGOT_ALT', 'gamma_GTP': 'gamma_GTP', 'SMK_stat_type_cd': 'Raucher_Status', 'DRK_YN': 'Trinker'})

dataset

Erkentnisse:

--> Input Josua (+ Team)

2. Daten transformieren:

In [ ]:
# Spalten transfomieren:

    # Trinker-Spalte: Y (für Yes) zu 1 & n (für No) zu 0

dataset["Trinker"] = dataset["Trinker"] == 'Y'
dataset["Trinker"] = dataset["Trinker"].astype(int)

    # Hörkraft-Spalten: 1 (für normal) bleibt 1 & 2 (für abnormal) zu 0

dataset["Hörkraft_links"] = dataset["Hörkraft_links"] == 1
dataset["Hörkraft_links"] = dataset["Hörkraft_links"].astype(int)

dataset["Hörkraft_rechts"] = dataset["Hörkraft_rechts"] == 1
dataset["Hörkraft_rechts"] = dataset["Hörkraft_rechts"].astype(int)

    # Hinzufügen einer berechneten Spalte - der Body-Mass-Index:

body_mass_index = dataset["Gewicht"].astype(np.float64) / ((dataset["Größe"].astype(np.float64) / 100) * (dataset["Größe"].astype(np.float64) / 100))

dataset["Body-Mass-Index"] = round(body_mass_index, 3)

    # Hinzufügen einer berechneten Spalte - der mittlere arterielle Blutdruck:

MAD = dataset["Diastolischer Blutdruck"] + (( dataset["Systolischer Blutdruck"] - dataset["Diastolischer Blutdruck"] ) * 0.5)

dataset["Mittlerer arterieller Blutdruck"] = MAD


dataset.head()

Zusammenfassung:

--> Input Team

3. Datenexploration:

In [ ]:
# Datenexploration zur Erkennung von Bereinigungsbedarf:

# Funktion zum Darstellen der Verteilung der Werte einer Spalte:

def bar_chart(column):
    share = (dataset[column].sort_values().value_counts(sort=False) / len(dataset)) * 100 
    plt.figure(figsize=(10, 6)) 
    share.plot(kind='bar') 
    plt.xlabel(share) 
    plt.ylabel('Prozentuale Häufigkeit')
    plt.grid(axis='y') 
    plt.show()

In [ ]:
column_names = dataset.columns.values.tolist()

for column in column_names:
    bar_chart(column)

Auffällige Datenpunkte:

1. Hüftumfang: Wenige Datenpunkte bei 999 cm
2. Sehkraft_links: Wenige Datenpunkte bei 9.9 (Verteilung der anderen Werte von 0,1 bis 2,0)
3. Sehkraft_rechts: Wenige Datenpunkte bei 9.9 (Verteilung der anderen Werte von 0,1 bis 2,0)

In [ ]:
# Korrelation:

def corr(columns):
    plt.figure(figsize=(8, 8))
    data = dataset[columns]
    heatmap = sns.heatmap(data.corr(), annot=True, cmap="coolwarm")


corr(['Body-Mass-Index','Gewicht','Größe'])

# Datenbereinigung

In [ ]:
# Es gibt 57 Reihen, in denen ein Hüftumfang von 9,99 m angegeben wurde:

filtered_dataset = dataset[dataset['Hüftumfang'] == 999]
len(filtered_dataset.index)

    # Eliminierung der fehlerhaften Reihen:

dataset = dataset[dataset['Hüftumfang'] != 999]



# Es gibt mehrer Reihen, in denen die Sehkraft 9,9 angegeben wurde. Dies bedeutet, dass der Proband erblindet ist.

filtered_dataset = dataset[dataset['Sehkraft_rechts'] == 9.9]
len(filtered_dataset.index)

filtered_dataset = dataset[dataset['Sehkraft_links'] == 9.9]
len(filtered_dataset.index)


dataset.describe()

Anmerkungen: 
